In [23]:
import duckdb
import pandas
import pyarrow as pa
import pyarrow.dataset as ds

In [2]:

%load_ext sql
%config SqlMagic.autopandas = False
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [4]:
df_games = duckdb.query("SELECT * FROM 'games.csv'")

In [6]:
%%sql
DESCRIBE select * from df_games

column_name,column_type,null,key,default,extra
column00,BIGINT,YES,None,None,None
Title,VARCHAR,YES,None,None,None
Release Date,VARCHAR,YES,None,None,None
Team,VARCHAR,YES,None,None,None
Rating,DOUBLE,YES,None,None,None
Times Listed,VARCHAR,YES,None,None,None
Number of Reviews,VARCHAR,YES,None,None,None
Genres,VARCHAR,YES,None,None,None
Summary,VARCHAR,YES,None,None,None
Reviews,VARCHAR,YES,None,None,None


In [10]:
df_games_processed =  duckdb.query("""SELECT *,
                 RIGHT(\"Release Date\",4) as Year,
                 CASE WHEN contains(Plays, 'K') THEN
                    CAST(replace(Plays, 'K', '') as FLOAT)*1000
                    ELSE
                    CAST(Plays AS FLOAT)
                    END as Plays_Float
                 FROM df_games""")

In [31]:
%%sql
DESCRIBE select * from df_games_processed

column_name,column_type,null,key,default,extra
column00,BIGINT,YES,None,None,None
Title,VARCHAR,YES,None,None,None
Release Date,VARCHAR,YES,None,None,None
Team,VARCHAR,YES,None,None,None
Rating,DOUBLE,YES,None,None,None
Times Listed,VARCHAR,YES,None,None,None
Number of Reviews,VARCHAR,YES,None,None,None
Genres,VARCHAR,YES,None,None,None
Summary,VARCHAR,YES,None,None,None
Reviews,VARCHAR,YES,None,None,None


In [20]:
%%sql
-- Top 10 títulos más jugados
SELECT Title, 
SUM(Plays_Float) as total_plays FROM 
                    df_games_processed                  
                    group by title
                    order by total_plays desc
                    LIMIT 10

Title,total_plays
Minecraft,110000.0
The Legend of Zelda: Breath of the Wild,90000.0
Grand Theft Auto V,90000.0
Doom,89300.0
Portal 2,87000.0
Undertale,84000.0
Portal,84000.0
Among Us,75000.0
Super Mario Odyssey,75000.0
Super Smash Bros. Ultimate,75000.0


In [22]:
%%sql
-- Listado de los juegos populares por año
SELECT DISTINCT Year, 
                  first(Title)  OVER(partition by Year order by SUM(Plays_Float) desc) as title, 
                  max(SUM(Plays_Float)) OVER(partition by Year order by SUM(Plays_Float) desc) as total_plays
                  FROM 
                    df_games_processed  
                    where trim(year) != 'TBD' and year <= '2023'         
                    group by Year, title                               
                    order by Year desc


Year,title,total_plays
2023,Hi-Fi Rush,9000.0
2022,Elden Ring,51000.0
2021,Resident Evil Village,29700.0
2020,Genshin Impact,42000.0
2019,Hades,63000.0
2018,Among Us,75000.0
2017,The Legend of Zelda: Breath of the Wild,90000.0
2016,Stardew Valley,60000.0
2015,Undertale,84000.0
2014,Chrome Dino,42000.0


In [43]:
part = ds.partitioning(
    pa.schema([("Year", pa.string())]), flavor="hive"
)

In [42]:
df_games_processed.to_arrow_table()

pyarrow.Table
column00: int64
Title: string
Release Date: string
Team: string
Rating: double
Times Listed: string
Number of Reviews: string
Genres: string
Summary: string
Reviews: string
Plays: string
Playing: string
Backlogs: string
Wishlist: string
Year: string
Plays_Float: float
----
column00: [[0,1,2,3,4,...,1507,1508,1509,1510,1511]]
Title: [["Elden Ring","Hades","The Legend of Zelda: Breath of the Wild","Undertale","Hollow Knight",...,"Back to the Future: The Game","Team Sonic Racing","Dragon's Dogma","Baldur's Gate 3","The LEGO Movie Videogame"]]
Release Date: [["Feb 25, 2022","Dec 10, 2019","Mar 03, 2017","Sep 15, 2015","Feb 24, 2017",...,"Dec 22, 2010","May 21, 2019","May 22, 2012","Oct 06, 2020","Feb 04, 2014"]]
Team: [["['Bandai Namco Entertainment', 'FromSoftware']","['Supergiant Games']","['Nintendo', 'Nintendo EPD Production Group No. 3']","['tobyfox', '8-4']","['Team Cherry']",...,"['Telltale Games']","['Sumo Digital', 'Sega']","['Capcom']","['Larian Studios']","['WB Gam

In [45]:
ds.write_dataset(df_games_processed.to_arrow_table(),"./output/videogames", format="parquet", partitioning=part, existing_data_behavior='overwrite_or_ignore')

In [50]:
games_from_parquet = ds.dataset("./output/videogames", format="parquet", partitioning=part)

In [52]:
%%sql
df_result << select * from games_from_parquet where Year in ('1985','1988')